In [1]:
from IPython.display import display
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

import datetime
from pathlib import Path

import intake

from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report, accuracy_score

import pickle

import tensorflow as tf
from tensorflow.keras import layers

2024-04-13 15:55:51.297297: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 15:55:52.717943: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
catalog = intake.open_catalog(Path('../catalog.yml'))
source = getattr(catalog, 'treesat')
gdf = source.read()[source.metadata['usecols']]
gdf.crs = 25832
gdf = gdf.to_crs(epsg=4326)

In [3]:
selected_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']

In [10]:
class Conv2DModelCreator:        
    def split_and_preprocess(self, y, X, random_state=42):
        """Split and max scale."""
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.1, random_state=random_state)
        
        train_band_max_values = [X_train[...,i].max() for i in range(X_train.shape[-1])]
        
        X_train = X_train/train_band_max_values

        test_band_max_values = [X_test[...,i].max() for i in range(X_test.shape[-1])]
        
        X_test = X_test/test_band_max_values

        return X_train, X_test, y_train, y_test
        
        
    def build_model(self, num_classes):
        padding = 'same'
        activation = 'relu'
        model = tf.keras.Sequential([
          layers.Conv2D(16, (3, 3), padding=padding, activation=activation),
          layers.MaxPooling2D(padding=padding),
          layers.Conv2D(32, (3, 3), padding=padding, activation=activation),
          layers.MaxPooling2D(padding=padding),
          layers.Conv2D(64, (3, 3), padding=padding, activation=activation),
          layers.MaxPooling2D(pool_size=2),
          layers.Flatten(),
          layers.Dense(128, activation=activation),
          layers.Dense(num_classes)
        ])

        model.compile(
            optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )

        return model
    
    def fit_eval(self, y, X, array_key, model):
        X_train, X_test, y_train, y_test = self.split_and_preprocess(y, X)
        
    
    def run(self, labels, filepaths):
        for filepath in filepaths:
            with open(filepath, 'rb') as f:
                X = np.load(f)
            # X = X[...,:3]

            X_train, X_test, y_train, y_test = self.split_and_preprocess(labels, X)
            
            print(f'Evaluating {filepath.stem}...', end=' ')

            model_name = f'conv2d_{filepath.parent}_{filepath.stem}'
            model_path = Path('models').joinpath(model_name)
            if model_path.is_file():
                model = joblib.load(model_path)
            else:
                model = self.build_model(labels.unique().size)
                model.fit(X_train, y_train, epochs=10, verbose=1)
                joblib.dump(model, model_path)
            
            model.evaluate(X_test, y_test, verbose=1)

In [11]:
target = source.metadata['categories']['multi'] # multi / trinary
gdf[target] = gdf[target].astype('category')

In [12]:
%%time
filepaths = sorted(list(Path('processed_data').glob('treesat_*.npy')))
zeros = Conv2DModelCreator().run(
    gdf[target].cat.codes, filepaths)

CPU times: user 334 µs, sys: 19 µs, total: 353 µs
Wall time: 350 µs
